#### 네이버 API Search Place 중단에 따라 카카오 API 로  실습코드 변경하였습니다 #####

- 안내 페이지: http://bit.ly/pwd_kakaoAPI_guide
- 깃헙 주소: https://github.com/Play-with-data/datasalon

# 5.3 지도시각화

###  5.3.2 데이터 준비

In [1]:
# 예제 5-24 크롤링 데이터 불러오기
import pandas as pd
raw_total = pd.read_excel('./files/1_crawling_raw.xlsx')
raw_total.head()

,content,date,like,place,tags
0,"제주에서 맞는 새해 첫날, 해는 못봤지만 눈은 원없이 보고 왔다!!! 눈되게 좋아하...",2019-01-01,431,1100고지,['#제주도']
1,저번에 못간 #동백군락지 🌺🌺보는것보다 사진이 더 이뻐요! (칭찬인지 욕인지.. 😜...,2019-01-01,"1,521",위미동백나무군락,"['#동백군락지', '#제주위미동백군락지', '#제주', '#위미', '#동백', ..."
2,.엘사는 아니지만 눈의 왕국을 다녀왔도다~~^^1100고지 주차장 잇바이데쓰 ㅠㅠ그...,2018-12-31,742,NaN,"['#제주', '#숲길', '#제주도', '#제주인생사진', '#제주여행', '#제..."
3,"#제주 #친봉산장 #adieu #2018 12월 31일,우리의 2018 마지막 데이...",2019-01-01,108,친봉산장,"['#제주', '#친봉산장', '#adieu', '#2018', '#럽스타그램', ..."
4,_와... 제주 겨울 바다 예술 😍😍😍,2019-01-01,718,Jeju Island 제주특별자치도 济州道,[]


In [2]:
# 예제 5-25 위치정보 가져오기
location_counts = raw_total['place'].value_counts( )
location_counts

Jeju                    190
Jeju-do                 140
Jungle Book by Alice    101
제주도 크리스마스 박물관            59
Seogwipo                 57
                       ... 
날것                        1
삼미흑돼지                     1
핸드테일러 제주점                 1
카페어떤오후                    1
큰갯물횟집                     1
Name: place, Length: 877, dtype: int64

In [4]:
# 예제 5-26 등록된 위치정보별 빈도수 데이터
location_counts_df = pd.DataFrame(location_counts)
location_counts_df.head()

,place
Jeju,190
Jeju-do,140
Jungle Book by Alice,101
제주도 크리스마스 박물관,59
Seogwipo,57


In [5]:
# 예제 5-27 위치정보 빈도수 데이터 저장하기
location_counts_df.to_excel('./files/3_location_counts.xlsx')

In [6]:
# 예제 5-28 위치정보 종류 확인하기
locations = list( location_counts.index )
locations 

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 '제주도 크리스마스 박물관',
 'Seogwipo',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 'Nimome',
 '1100고지',
 '월정리해변',
 '석부작박물관',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '제주도 애월읍',
 '제주 함덕 서우봉 해변',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '할로비치',
 '위미동백나무군락지',
 '연동 바오젠 거리',
 '제주신화월드 Jeju Shinhwa World',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '새별오름',
 'Woljeongri Beach',
 '협재해변 Beach',
 '고래배꼽',
 '고집돌우럭중문점',
 '용눈이오름',
 '제주빅볼랜드',
 '밥깡패',
 '사려니숲길',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '삼무공원',
 '휴애리 자연생활공원',
 '마마뜰',
 '위미2리 동백군락지',
 '제주커피박물관 Baum',
 '캔디원',
 '서귀포 느영나영 게스트하우스',
 '한라산',
 '용머리해안',
 'Jeju Aewol',
 '광치기해변',
 '한라산 (漢拏山, Hallasan)',
 '하이엔드 제주',
 '서우봉',
 '옹포별장가든',
 '성이시돌목장',
 'Jeju Island 제주특별자치도 济州道',
 '제주 송악산',
 '위미동백나무군락',
 '제주해남',
 '월정리카페콧수염',
 '산굼부리',
 '카페브리프',
 '에코랜드',
 '牛岛 Udo Island 우도',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 '카페한라산',
 '곽지해수욕장',
 '제주어린왕자게스트하우스',
 '우드노트',
 '번개과학체험관',
 '제주신라호텔',
 '르페도라펜션',
 'Terarosa - 테라로사

### 5.3.3 카카오 검색  API를 활용한 장소 검색

In [8]:
# 예제 5-29 카카오 검색 API 사용 예시

import requests
with open('private','r') as f:
    for i in f.readlines():
        mykey = i.strip()
searching = '합정 스타벅스'
url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)

headers = {
    "Authorization": f"KakaoAK {mykey}"
    # 입력시 반드시  KakaoAK 뒤에 한 칸 띄우고 API키 값을 적어야 합니다. 
}

places = requests.get(url, headers = headers).json()['documents']
places



[{'address_name': '서울 마포구 서교동 395-166',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '26572121',
  'phone': '1522-3232',
  'place_name': '스타벅스 서교점',
  'place_url': 'http://place.map.kakao.com/26572121',
  'road_address_name': '서울 마포구 양화로 78',
  'x': '126.916980454434',
  'y': '37.5514601750423'},
 {'address_name': '서울 마포구 합정동 472',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '2057327896',
  'phone': '1522-3232',
  'place_name': '스타벅스 합정점',
  'place_url': 'http://place.map.kakao.com/2057327896',
  'road_address_name': '서울 마포구 월드컵로1길 14',
  'x': '126.91253700818196',
  'y': '37.54994959743763'},
 {'address_name': '서울 마포구 서교동 490',
  'category_group_code': 'CE7',
  'category_group_name': '카페',
  'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
  'distance': '',
  'id': '288597324',
  'phone': '1522-3232',
  'place_n

In [9]:
# 예제 5-30 카카오 로컬 API를 활용한 장소 검색 함수 만들기
def find_places(searching):
    # ① 접속URL 만들기
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    # ② headers 입력하기
    headers = {
        "Authorization": f"KakaoAK {mykey}"
    }
    # ③ API 요청&정보 받기
    places = requests.get(url, headers = headers).json()['documents']
    # ④ 필요한 정보 선택하기
    place = places[0] 
    name = place['place_name']
    x=place['x']
    y=place['y']
    data = [name, x, y, searching] 

    return data


In [10]:
# 예제 5-31 제주공항 검색 예시
data = find_places('제주공항')
data

['제주국제공항', '126.492769004244', '33.5070789578184', '제주공항']

In [11]:
# 반복작업 진행시 진행바 표시하기위한 라이브러리 tqdm 활용하기
# ! pip install tqdm   # tqdm 라이브러리 설치하기
from tqdm import tqdm

In [12]:
# 예제 5-32 인스타그램 위치명 위치정보 검색하기
#### 검색시 시간이 많이 걸려, 현재 작업 진행현황을 볼 수 있도록,  tqdm_notebook 추가 했습니다.

import time
locations_inform = [ ]
for location in tqdm(locations):
    try:
        data = find_places(location)       
        locations_inform.append(data) 
        time.sleep(0.1) 
    except:
        pass
locations_inform


100%|██████████| 877/877 [02:12<00:00,  6.62it/s]


[['Jeju in aA', '126.83650369902168', '33.53833652360498', 'Jeju'],
 ['바이나흐튼 크리스마스박물관', '126.32785786016811', '33.2915720831759', '제주도 크리스마스 박물관'],
 ['서귀포잠수함', '126.558616052674', '33.2393033784206', 'Seogwipo'],
 ['제주에인감귤밭', '126.5390139270361', '33.25656069063887', '제주에인감귤밭'],
 ['제주도', '126.54587355630036', '33.379777816446165', 'Jeju Island'],
 ['한라산1100고지', '126.462219691112', '33.3580781709788', '1100고지'],
 ['월정리해수욕장', '126.795805057888', '33.556469394054', '월정리해변'],
 ['석부작박물관', '126.53652102219156', '33.25045551460648', '석부작박물관'],
 ['알뜨르비행장', '126.271527320164', '33.2047161778831', '알뜨르 비행장'],
 ['카멜리아힐', '126.370009467525', '33.2896344311608', '카멜리아 힐'],
 ['제주레포츠랜드', '126.6387551242765', '33.48206819907232', '제주레포츠랜드'],
 ['올레길 16코스(고내-광령 올레)', '126.38979561557086', '33.46649759191228', '제주도 애월읍'],
 ['함덕해수욕장', '126.669238934013', '33.5430615661113', '제주 함덕 서우봉 해변'],
 ['할로비치 본점', '126.2815248933638', '33.30832616522123', '할로비치'],
 ['위미동백나무군락', '126.67473058287', '33.2729387205764',

In [13]:
# 예제 5-33 위치정보 저장하기
locations_inform_df = pd.DataFrame(locations_inform)
locations_inform_df.columns = ['name_official','경도','위도','인스타위치명']
locations_inform_df.to_excel('./files/3_locations.xlsx', index=False)


In [14]:
# 예제 5-34 인스타 게시량 및 위치정보 데이터 불러오기
location_counts_df = pd.read_excel('./files/3_location_counts.xlsx', index_col = 0)
locations_inform_df = pd.read_excel('./files/3_locations.xlsx')

In [15]:
# 예제 5-35 위치 데이터 병합하기
location_data = pd.merge(locations_inform_df, location_counts_df, 
                         how = 'inner', left_on = 'name_official', right_index=True)

location_data.head()


,name_official,경도,위도,인스타위치명,place
2,서귀포잠수함,126.558616,33.239303,Seogwipo,1
447,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
3,제주에인감귤밭,126.539014,33.256561,제주에인감귤밭,48
4,제주도,126.545874,33.379778,Jeju Island,3
112,제주도,126.545874,33.379778,제주도,3


In [16]:
# 예제 5-36 데이터 중복 점검하기
location_data['name_official'].value_counts()

제주동문재래시장    3
오설록티뮤지엄     3
서귀포잠수함      2
도두항         2
천제연폭포       2
           ..
북촌플레이스      1
월정곰닭        1
연화키친        1
돈사촌         1
큰갯물횟집       1
Name: name_official, Length: 318, dtype: int64

In [17]:
# 예제 5-37 장소 이름 기준 병합하기
location_data = location_data.pivot_table(index = ['name_official','경도','위도'], values = 'place', aggfunc='sum')
location_data.head()

,,,place
name_official,경도,위도,
73st,126.455109,33.456989,1
가드망제,126.532954,33.500991,1
가배,126.951868,37.738173,1
가시아방국수,126.918065,33.438605,2
객의하우스,126.243342,33.396697,1


In [18]:
# 예제 5-38 병합한 데이터 저장하기
location_data.to_excel('./files/3_location_inform.xlsx')

### 5.3.6 folium을 이용한 지도 시각화 ① - 개별 표시

In [19]:
# 예제 5-39 데이터 불러오기
location_data = pd.read_excel('./files/3_location_inform.xlsx')
location_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  318 non-null    object 
 1   경도             319 non-null    float64
 2   위도             319 non-null    float64
 3   place          319 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 10.1+ KB


In [20]:
# 예제 5-40 지도 표시하기
### 변수명 수정  수정일자 2020.02.17

import folium

Mt_Hanla =[33.362500, 126.533694]
map_jeju = folium.Map(location = Mt_Hanla, zoom_start = 11)

for i in range(len(location_data)):
    name = location_data ['name_official'][i]    # 공식명칭
    count = location_data ['place'][i]           # 게시글 개수
    size = int(count)*2
    long = float(location_data['위도'][i])      
    lat = float(location_data['경도'][i])       
    folium.CircleMarker((long,lat), radius = size, color='red', popup=name).add_to(map_jeju)
    
map_jeju

In [21]:
# 예제 5-41 지도 저장하기
map_jeju.save('./files/3_jeju.html') 

### 4.3.7 folium을 이용한 지도 시각화 ② - 그룹으로 표시

In [22]:
# 예제 5-42 지도 표시하기(마커 집합)
### 변수명 수정  수정일자 2020.02.17

from folium.plugins import MarkerCluster

locations = []
names = []

for i in range(len(location_data)):
    data = location_data.iloc[i]  # 행 하나씩
    locations.append((float(data['위도']),float(data['경도'])))    # 위도 , 경도 순으로..
    names.append(data['name_official'])


Mt_Hanla =[33.362500, 126.533694]
map_jeju2 = folium.Map(location = Mt_Hanla, zoom_start = 11)
                       
marker_cluster = MarkerCluster(
    locations=locations, popups=names,
    name='Jeju',
    overlay=True,
    control=True,

)

marker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)

map_jeju2


In [23]:
# 예제 5-43 지도 저장하기
map_jeju2.save('./files/3_jeju_cluster.html') 